## Graphs example

This notebook demonstrates some of the functionality of topos-tool in the example of the category of graphs (i.e. directed multigraphs).
We start by defining a basis category.
Note that we call this one "Graphs" but it is the category of presheaves on it that are graphs.
Also note that this particular example can be loaded by typing `open Examples.Graphs` but we make it manually here for demo purposes.

First load the library `ToposTool.dll` and the `Latex.print` command.

In [1]:
#r "../bin/Release/net5.0/ToposTool.dll"
#load "../scripts/LatexPrint.fsx"

open LatexPrint

The below script needs to be able to find the current output cell; this is an easy method to get it.

Next we make the basis category.

We start by defining a type for the objects of the category as a single-case discriminated union of string type. (Defining our objects as plain strings, or of any other type would also work, but doing it this way ensures the type system will save us from accidentally combining structures on different categories in invalid ways).

Next we will define the objects, arrows and composition relation of the category.

The basis category has two objects $V$ and $E$ representing the vertex and edge.

We then define the arrows of the category. Here there are two, $s$ (source) and $t$ (target), both going from the vertex to the edge.

Last we define the composition relation. In this case there are no nontrivial data.
Now we build the category using `Category.make` which automatically fills in the trivial data.

The category looks like this:
$$V \mathrel{\mathop{\rightrightarrows}^{\mathrm{s}}_{\mathrm{t}}} E$$

We can display it using `Latex.print`.

In [2]:
type Graphs = Graphs of string

let V, E = Graphs "V", Graphs "E"
let objects = [ V; E ]

let s, t = Arrow.make "s" V E, Arrow.make "t" V E
let arrows = [ s; t ]

let compose = []

let cat = Category.make "Graphs" objects arrows compose

cat |> Latex.print

---
Next we instantiate the Yoneda functor
$$ h : \mathsf{C} \to \mathsf{Set^{C^{op}}}$$
whose action on objects is
$$ A \mapsto \mathsf{hom} \langle - , A\rangle$$

The image of the Yoneda functor on the objects of our basis are the representable presheaves.

In [3]:
let yo = Yoneda.yo cat

let hV, hE = yo.Ob.[V], yo.Ob.[E]

As a graph, hV is a single vertex:

In [4]:
hV |> Latex.print

And hE is a graph with two vertices and a single arrow between them:

In [5]:
hE |> Latex.print

---
We can perform some basic constructions on subgraphs with products using `*` (a synonym for `Presheaf.product`), sums using `+` (a synonym for `Presheaf.sum`) and exponentials using `^` (a synonym for `Presheaf.exp`), and we can verify identities using `==` (a synonym for `Presheaf.isIso`).

Below we verify that
$$ h_E \times h_E \simeq h_E + h_V + h_V$$

In [6]:
hE * hE == hE + hV + hV

True

The graph exponential is defined via the adjunction
$$ \mathsf{hom} \langle F \times G, H \rangle \simeq \mathsf{hom} \langle F , H^G \rangle$$

Below we compute $${h_E}^{h_V}$$ which is a graph with 2 vertices and 4 edges--two of them loops on the vertices and the other two going from one vertex to the other in opposite directions.

In [7]:
hE ^ hV |> Latex.print

---
The generated names may be complicated, but we use `Presheaf.simplify` to generate an isomorphic presheaf whose figures are integers.

In [8]:
let F = hE ^ hV |> Presheaf.simplify

F |> Latex.print

In [9]:
F == hE ^ hV

True

---
The algebra of subobjects of a presheaf is computed with `Subobject.algebra`.
We can check that there are 21 subgraphs of our last example.

In [10]:
let alg = Subobject.algebra F

alg.Subobjects |> Set.count 

21

The truth object is a graph for which morphisms to it correspond to subobjects:
$$ \mathsf{sub\,F} \simeq \mathsf{hom} \langle F, \Omega \rangle$$

We can compute the truth object of a category using `Truth.omega` and morphisms between presheaves using `Morphism.hom`.

In [11]:
let Om = Truth.omega cat

Morphism.hom F Om |> Set.count

21

The subobjects have the structure of a biheyting algebra that supports operations like meet, join, implication, subtraction, negation, supplement.

We will take an arbitrary subgraph of the previous example. It happens to have two vertices and no edges:

In [12]:
let g = alg.Subobjects |> Seq.item 2 

g |> Latex.print

The negation of a subgraph is the largest subgraph disjoint from it. For $g$ above, that is the empty graph (or the zero object `Presheaf.zero`):

In [13]:
let neg_g = Subobject.negate alg g

neg_g == Presheaf.zero cat

True

The boundary of a subgraph $X$ is the subgraph of vertices of $X$ connected to the outside.

We will check the product rule for the boundary operator holds on the algebra

$$ \partial (X \land Y) = (\partial X \land Y) \lor ( X \land \partial Y)$$

In [14]:
let (.+) = Subobject.join
let (.*) = Subobject.meet
let d = Subobject.boundary alg

let productRule (X, Y) = d (X .* Y) = (d X .* Y) .+ (X .* d Y)

alg.Subobjects |> Set.square |> Set.forall productRule

True

---
Graphs can be glued together using colimits. We will glue the two vertices of hE to make a loop.

First we find the graph morphisms from $h_V$ to $h_E$: there are just two, sending the single vertex of hV to either vertex of hE.

In [15]:
let morphisms = Morphism.hom hV hE

morphisms |> Latex.print

To glue the two vertices of $h_E$ together, we use `Presheaf.coequaliser` to take the coequaliser of the diagram
$$h_V \mathrel{\mathop{\rightrightarrows}^{\mathrm{n}}_{\mathrm{m}}} h_E$$

We check it is a loop by printing its data.

In [16]:
let f = morphisms |> Seq.item 0
let g = morphisms |> Seq.item 1

let L = Presheaf.coequaliser f g

L |> Latex.print